In [1]:
#pytorch
# cv2
# matplotlin
# conda install conda-forge::tqdm
# pip install dill
# pip cache purge
# pip install git+https://github.com/THU-MIG/yolov10.git
# pip install huggingface-hub

In [1]:
from torch import nn
from ultralytics import YOLO

import os
import json 

In [2]:
class Constants_Models_Weights:
    YOLOV3 = "yolov3n.pt"
    YOLOV5 = "yolov5nu.pt"
    YOLOV6 = "yolov6-n.pt"
    YOLOV8 = "yolov8n.pt"
    YOLOV9 = "yolov9c.pt"

class Constants_Models:
    YOLOV3 = "yolov3"
    YOLOV5 = "yolov5"
    YOLOV6 = "yolov6"
    YOLOV8 = "yolov8"
    YOLOV9 = "yolov9"

class Constants_Training:
    PRETRAINED = "pretrained"
    FROM_SCRATCH = "scratch"
    FINE_TUNING = "fine-tuning"

In [3]:
def save_json(data, filename):
    if os.path.isfile(filename):
        os.remove(filename)
    json.dump(data, open(filename, 'w'), indent=4)

In [4]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

In [5]:
class Yolov9(nn.Module):
    
    def __init__(self, args):
        super(Yolov9, self).__init__()

        # For RGB Datasets
        if args["training_mode"] == "pretrained":
            self.model = YOLO("yolov9c.pt")
            self.model.add_callback("on_train_start", freeze_layer)
        elif args["training_mode"] == "scratch":
            self.model = YOLO("C:/Users/dnnxl/Documents/multispectral_yolov9/models/yolov9.yaml")
        elif args["training_mode"]== "fine-tuning":
            self.model = YOLO("yolov9c.pt")
        
    def train(self, args, output_folder):

        train_results = self.model.train(data=args["data_root"], epochs=args["epochs"], imgsz=args["imgsz"], patience=args["patience"], batch=args["batch"], 
              save=args["save"], save_period=args["save_period"], device=args["device"], workers=args["workers"], project=output_folder, 
              name=args["name"], pretrained=args["pretrained"], optimizer=args["optimizer"], seed=args["seed"], cos_lr=args["cos_lr"], 
              resume=args["resume"], lr0=args["lr0"], momentum=args["momentum"], weight_decay=args["weight_decay"], val=args["val"])
        
        return train_results 


    def test(self, args):
        test_results = self.model.val(data=args["data_root"], device=args["device"])
        return test_results

In [6]:
def yolo_v(args, output_root):
    
    # Create the output directory
    if not os.path.exists(output_root):
        os.makedirs(output_root, exist_ok=True)

    # STEP 1: Save the training configuration in a file 
    save_json(args, os.path.join(output_root, "train_config.json"))
    # STEP 2: Save
    if args["model"] == Constants_Models.YOLOV9:
        model = Yolov9(args)

    # STEP 3: Train the model
    train_results = model.train(args, output_folder=output_root)
    save_json(train_results.results_dict, os.path.join(output_root, "train_results.json"))
    # STEP 4: Test the model on the test dataset
    test_results = model.test(args)
    save_json(test_results.results_dict, os.path.join(output_root, "test_results.json"))

    return test_results.results_dict

In [7]:
# Define the arguments manually
args = {
    'data_root': "C:/Users/dnnxl/Documents/pineapple_yolov9/pineapples_1.v1i.yolov9/data.yaml",
    'output_root': './output/',
    'epochs': 2,
    'patience': 100,
    'batch': 16,
    'imgsz': 160,
    'save_period': -1,
    'device': "cpu",
    'workers': 4,
    'name': None,
    'pretrained': 1,
    'optimizer': 'auto',
    'seed': 0,
    'resume': 0,
    'lr0': 0.001,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'model': 'yolov9',
    'model_weights': 'yolov10n.pt',
    'numa': -1,
    'training_mode': "fine-tuning",  # pretrained, scratch, fine-tuning
    'model': 'yolov9',
    'save': True,
    'cos_lr': True, 
    'val': False
}

In [8]:
yolo_v(args, './output/')

Ultralytics YOLOv8.2.32  Python-3.12.3 torch-2.3.1 CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=yolov9c.pt, data=C:/Users/dnnxl/Documents/pineapple_yolov9/pineapples_1.v1i.yolov9/data.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=160, save=True, save_period=-1, cache=False, device=cpu, workers=4, project=./output/, name=train, exist_ok=False, pretrained=1, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=0, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

train: Scanning C:\Users\dnnxl\Documents\pineapple_yolov9\pineapples_1.v1i.yolov9\train\labels.cache... 124 images, 0 backgrounds, 0 corrupt: 100%|██████████| 124/124 [00:00<?, ?it/s]
val: Scanning C:\Users\dnnxl\Documents\pineapple_yolov9\pineapples_1.v1i.yolov9\valid\labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]


Plotting labels to output\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
Image sizes 160 train, 160 val
Using 0 dataloader workers
Logging results to output\train
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G        5.4      5.115      1.446        896        160: 100%|██████████| 8/8 [00:42<00:00,  5.34s/it]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


                   all         31       1420      0.168     0.0479     0.0349     0.0109
Speed: 0.6ms preprocess, 68.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to output\train
Ultralytics YOLOv8.2.32  Python-3.12.3 torch-2.3.1 CPU (12th Gen Intel Core(TM) i5-1235U)
YOLOv9c summary (fused): 384 layers, 25320019 parameters, 0 gradients, 102.3 GFLOPs


val: Scanning C:\Users\dnnxl\Documents\pineapple_yolov9\pineapples_1.v1i.yolov9\valid\labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]


                   all         31       1420      0.168     0.0479     0.0349     0.0109
Speed: 0.1ms preprocess, 69.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to output\train2


{'metrics/precision(B)': 0.16843778753149238,
 'metrics/recall(B)': 0.04788732394366197,
 'metrics/mAP50(B)': 0.03486571694632599,
 'metrics/mAP50-95(B)': 0.010903269288814802,
 'fitness': 0.013299514054565921}